In [46]:
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
import datetime
import os
from itertools import islice
from collections import defaultdict
import networkx as nx
import nltk
import pickle
from community import community_louvain

In [20]:
data_comments = pd.read_csv(os.getcwd() + '/comments.csv')
data_submission = pd.read_csv(os.getcwd() + '/submissions.csv')

In [60]:
data_comments.columns
unique_users = np.unique(np.array(data_comments['author']))
unique_subreddits = np.unique(np.array(data_comments['subreddit']))

In [84]:
num_posts_in_subreddit = defaultdict(lambda: 0)
num_comments_per_subreddit = {user: {subreddit: 0 for subreddit in unique_subreddits} for user in unique_users}
for _, comment in tqdm(data_comments.iterrows()):
    num_comments_per_subreddit[comment['author']][comment['subreddit']] += 1

287889it [00:09, 31945.98it/s]


In [85]:
#num_comments_per_subreddit[unique_users[50000]]['Oscars']
num_comments_per_subreddit

{'---------_----_---_': {'Oscars': 0,
  'OutOfTheLoop': 0,
  'USNEWS': 0,
  'boxoffice': 0,
  'entertainment': 0,
  'news': 1,
  'offbeat': 0,
  'qualitynews': 0,
  'television': 0,
  'willsmith': 0,
  'worldnews': 0},
 '-------I-------': {'Oscars': 0,
  'OutOfTheLoop': 0,
  'USNEWS': 0,
  'boxoffice': 0,
  'entertainment': 0,
  'news': 1,
  'offbeat': 0,
  'qualitynews': 0,
  'television': 0,
  'willsmith': 0,
  'worldnews': 0},
 '------sb': {'Oscars': 0,
  'OutOfTheLoop': 0,
  'USNEWS': 0,
  'boxoffice': 0,
  'entertainment': 0,
  'news': 2,
  'offbeat': 0,
  'qualitynews': 0,
  'television': 0,
  'willsmith': 0,
  'worldnews': 0},
 '-----0-----': {'Oscars': 1,
  'OutOfTheLoop': 0,
  'USNEWS': 0,
  'boxoffice': 0,
  'entertainment': 0,
  'news': 0,
  'offbeat': 0,
  'qualitynews': 0,
  'television': 0,
  'willsmith': 0,
  'worldnews': 0},
 '-----1': {'Oscars': 0,
  'OutOfTheLoop': 0,
  'USNEWS': 0,
  'boxoffice': 0,
  'entertainment': 0,
  'news': 0,
  'offbeat': 0,
  'qualitynews': 

In [21]:
# Dictionary to get author of a given comment based on id
get_author = {row['id']: row['author'] for _,row in tqdm(data_comments.iterrows())}
#[num_posts_in_subreddit[comment['author'][comment['subreddit']]] += 1 for comment in comments.iterrows()]
#max_subreddit_posts = {subreddit, user for user, subreddit in}

287889it [00:08, 32953.09it/s]


In [23]:
connections = defaultdict(lambda: int(0))
i=0
for _,row in tqdm(data_comments.iterrows()):
    try: connections[row['author'],get_author[row['parent_id'][3:]]] += 1 #3: since we must remove outgoing/incoming comment
    except KeyError:
        i += 1

        
print('number of removed comments due to errors', i)

        
edges_directed = [(row[0],row[1], connections[row[0],row[1]]) for row in connections if (row[0], row[1]) in connections]

# Old, non-directed graph
#edges_undirected = [(row[1],row[0], connections[row[1],row[0]] + connections[row[0],row[1]]) for row in connections if (row[0],row[1]) in connections and (row[1],row[0]) in connections]


G = nx.DiGraph()

for u,v,w in edges:
    G.add_edge(u,v,weight = w)

287889it [00:09, 31072.33it/s]


number of removed comments due to errors 81884


In [49]:
#nx.algorithms.community.louvain_partitions(G, weight='weight', resolution=1, threshold=1e-07, seed=None)
#nx.algorithms.community.louvain_partitions(G, weight='weight', resolution=1, threshold=1e-07, seed=None)

community_louvain.best_partition(G)

TypeError: Bad graph type, use only non directed graph